In [ ]:
# import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import balanced_accuracy_score, accuracy_score, roc_auc_score, f1_score

import pickle
from random import choices
from itertools import product

from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

### 1) Necessary functions

In [ ]:
# The function grid_generator generates a grid space used to find the next point to label based on uncertainty measures.
# refer to the paper for more info on the input variables.
def grid_generator():
    tool = ['TOOLING-Aluminium-606x-Null-v1', # Tool material
            'TOOLING-Composite-AS4-v1',
            'TOOLING-Steel-1020-Null-v1',
            'TOOLING-Invar-36-Null-v2']
    top_htc = np.round(np.linspace(10,200,10)) # Top side htc
    bottom_htc = np.round(np.linspace(5,150,10)) #  Bottom side htc
    tool_thick = np.round(np.linspace(1,30,10)) # Tool thickness (mm)
    mat_thick = np.round(np.linspace(1,30,10)) # Part thickness (mm)
    ramp1 = np.round(np.linspace(1,5,5)) # Ramp 1 heating rate (C/min)
    ramp2 = np.round(np.linspace(1,5,5)) # Ramp 2 heating rate (C/min)
    hold1 = np.round(np.linspace(105,125,5)) # Hold time (C)

    grid_np = np.array(list(product(tool, tool_thick, mat_thick, top_htc, bottom_htc, ramp1, hold1, ramp2)))
    inp_col_names = ['tool', 'Tool_Thickness', 'Part_Thickness', 'Top_HTC', 'Bottom_HTC','Ramp1', 'Hold1_Temp', 'Ramp2']
    grid = pd.DataFrame(grid_np, columns=inp_col_names)
    return grid

# The function takes the grid space and preprocess the data (one-hot encoding, normalization, etc.)
def grid_preprocess(grid):
    # One-hot encoder
    encoder = OneHotEncoder()
    toolMat = grid.iloc[:,0]
    toolMat_1hot = encoder.fit_transform(toolMat.values.reshape(-1, 1))
    toolMat_df = pd.DataFrame(toolMat_1hot.toarray(), columns=['ToolMat_1', 'ToolMat_2', 'ToolMat_3', 'ToolMat_4'])
    #data_1hot = pd.concat([toolMat_df, data], axis=1)
    data_concat = pd.concat([toolMat_df.reset_index(), grid.reset_index()], axis=1)
    data_concat.drop(['index', 'tool'], axis=1, inplace=True)
    
    # Scale the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_concat)
    
    # Create mapping for onehot encoder
    encoder_categories = ['TOOLING-Aluminium-606x-Null-v1', 'TOOLING-Composite-AS4-v1',
                                        'TOOLING-Steel-1020-Null-v1',
                                        'TOOLING-Invar-36-Null-v2']
    encode_cc1 = [1, 2, 3, 4]
    Tool_Mapping = {}
    for i, j in zip(encode_cc1, encoder_categories):
        Tool_Mapping[i] = j
    
    return data_scaled, scaler, Tool_Mapping, encoder

# The function takes the raw data and prepares it for the training process
def loadData(data, encoder):
    toolMat = np.array(data['Tool_Material'])
    toolMat_1hot = encoder.fit_transform(toolMat.reshape(-1, 1))
    toolMat_df = pd.DataFrame(toolMat_1hot.toarray(), columns=['ToolMat_1', 'ToolMat_2', 'ToolMat_3', 'ToolMat_4'])
    data_concat = pd.concat([toolMat_df.reset_index(), data.reset_index()], axis=1)
    data_concat.drop(['index', 'Tool_Material'], axis=1, inplace=True)
    data_concat.rename(columns={'Part_Max_Exotherm_Value': 'exotherm', 'Part_Max_Lag_Value': 'lag'}, inplace=True)
    return data_concat, encoder


### 2) Source network training

In [ ]:
# 1) Generate the grid space and preprocess the data
grid = grid_generator()
data_processed, scaler, Tool_Mapping, onehot_encoder = grid_preprocess(grid)

# 2) Load the one-hot encoder (alternatively one can use "onehot_encoder" from the code above).
f = open('encoder.pickle', 'rb')
onehot_encoder = pickle.load(f)

# 3) Read data (train and validation)
path = '\Data\\'
filename = '8552_Sample.csv'
Main = pd.read_csv(path + filename, index_col=0)
data, encoder = loadData(Main,onehot_encoder)
X_train, X_valid, y_train, y_valid, scaler_s = dataprep(data, 0.2, refScaler=scaler)

# 4) Source network architecture
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=[11]),
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(8, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(5, activation='selu', kernel_initializer='lecun_normal')
])
classifier = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'), 
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model = tf.keras.models.Sequential([encoder, classifier])

# 5) Training the source model
lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr=lr)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
patience = 100
cw = {0:1, 1:10}
callback = tf.keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True)
history_source = model.fit(x=X_train, y=y_train, epochs=1000, batch_size=128,
                                  validation_data=(X_valid, y_valid), callbacks=[callback], class_weight=cw)



### 3) Active learning via GP (RAVEN software as the oracle)

In [ ]:
##### Functions ######

# Create Variational GP model
class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Evaluate the model
def evaluate():
    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    means[means<0] = -1
    means[means>0] = 1

    metrics = []
    for i in [balanced_accuracy_score, accuracy_score, roc_auc_score, f1_score]:
        metrics.append(np.round(i(test_y, means),3))
    return metrics

# Quary the next point to be labeled based on acquisition functions
def active_query(n):
    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    stds = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in grid_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
            stds = torch.cat([stds, preds.stddev.cpu()])
    means = means[1:]
    stds = stds[1:]

    # Acquisition functions (AFs) --> Refer to the paper for more info
    
    ## AF1:
    #crit = (abs(means))/(torch.sqrt(1 + stds**2))
    #arg_query = torch.argsort(crit)[:n]
    
    ## AF2:
    #crit = stds**2
    #arg_query = torch.argsort(crit, descending=True)[:n]
    
    ## AF3:
    crit = abs(means)
    arg_query = torch.argsort(crit)[:n]
    
    return arg_query

# Generate inputs for RAVEN
def generate_input(arg_query, scaler, Tool_Mapping, onehot_encoder):
    inverse_norm = scaler.inverse_transform(data_processed[arg_query])
    tool = onehot_encoder.inverse_transform(inverse_norm[:,:4])
    return np.concatenate([tool.reshape(-1,1), inverse_norm[:,4:]], axis=1)

# Run RAVEN software (oracle) to label the quaried data
def RAVEN(data, n, m, resin, dire_update):
    import time
    import ravenAPI
    
    toolingMaterial, toolthicknessValue, partthicknessValue, topHtcValue, bottomHtcValue, ramp1, holdtemp, ramp2 = data
    
    if resin == 8552:
        part_mat = 'COMPOSITE-HEXCEL-8552-AS4-PW-v2'
    elif resin == 8551:
        part_mat = 'COMPOSITE-HEXCEL-8551-7-AS4-Tape-v1'

    # Create parametric input object
    inputs = ravenAPI.ParametricInputs()

    # Specify top and bottom HTCs (W/(m2 K))
    inputs.addTopHTC(topHtcValue)
    inputs.addBottomHTC(bottomHtcValue)

    # Specify tooling facesheet thicknesses (meters)
    inputs.addToolingThickness(toolthicknessValue/1000)

    # Specify tooling materials (by unique name, which should be the filename without the extension)
    for i in [toolingMaterial]:
        inputs.addToolingMaterial(i)

    # Create, populate, and add thermal cycles (C, C/min and min)
    newCycle = ravenAPI.ThermalCycle('2Hold_ramp1_{}_hold_{}_ramp2_{}'.format(str(ramp1), str(holdtemp), str(ramp2)), 20)  # cycle name, initial temp
    newCycle.addRamp(holdtemp, ramp1)  # target temp, heating rate
    newCycle.addHold(60)  # hold time
    newCycle.addRamp(180, ramp2)  # target temp, heating rate
    newCycle.addHold(120)  # hold time
    newCycle.addRamp(20, 3.5)  # target temp, heating rate
    inputs.addThermalCycle(newCycle)

    # Create, populate, and add material stacks
    newStack = ravenAPI.MaterialStack('Part_{}'.format(str(partthicknessValue)))  # stack name
    # material unique name, laminate thickness, initial DoC
    newStack.addLaminate(part_mat, partthicknessValue/1000, 0.001)
    inputs.addMaterialStack(newStack)

    # Submit analysis
    analysisID = ravenAPI.submitParametricAnalysis(inputs)
    print("Started analysis: %s" % analysisID)

    # Wait for the analysis to complete
    while ravenAPI.isRunning(analysisID):
        time.sleep(1)

    # Save results
    direc = dire_update
    ravenAPI.saveResultsCSV(analysisID,
                            direc + 'sample_{}_AL{}.csv'.format(str(n), str(m)),
                            fileFormat='hashable')
    return print('sample_{} done'.format(str(n)))

def load_main_data(dire, main, iter_n):
    """:param
    :param dire: directory of queried data from active learning procedure
    :param query_num: query number (sequence)
    :param main: main dataset so far (orig + data added from active learning)
    """
    import os 
    import numpy as np
    import pandas as pd
    #dire = r'C:\Users\miladrmz\iCloudDrive\PhD\VarGP_TL_Paper\Codes\Test\\'
    fileName = os.listdir(dire)
    data = None
    for i in fileName:
        
        if 'IN_' not in i:
        
            # read CSV file
            raw = pd.read_csv(dire + i)

            # Cure cycle specifications
            ccSpec = pd.DataFrame(np.array([[j[5], j[9:12], j[22]] for j in raw['Cure_Cycle_Name']]),
                                  columns=['Ramp1', 'Hold1_Temp', 'Ramp2'])

            # Input/output variables
            # Change units to mm so that it matches the rest of the code
            raw[['Tool_Thickness', 'Part_Thickness']] = raw[['Tool_Thickness', 'Part_Thickness']]*1000

            inptClmnName = ['Tool_Material', 'Tool_Thickness', 'Part_Thickness', 'Top_HTC', 'Bottom_HTC']
            inps = raw[inptClmnName]
            outps = raw[['Part_Max_Exotherm_Value','Part_Max_Lag_Value']]

            # Concat cure cycle specs and input variables in one DF
            temp_inp = pd.concat([inps,ccSpec], axis=1)
            temp = pd.concat([temp_inp,outps], axis=1)
            #temp = pd.concat([toolMat_df,temp_num], axis=1)

            # Create Final data table
            if data is None:
                data = temp
            else:
                data = pd.concat([data,temp], axis = 0).reset_index(drop=True)
                
            os.rename(dire + i, dire + 'IN_' + i)

    data.to_csv('8552_Main_Limited_iter{}.csv'.format(iter_n))

    ## adding queried data to the main dataset
    main_T = pd.concat([main,data], axis = 0).reset_index(drop=True)
    
    return main_T, data

In [ ]:
# 1) Generate the grid space and preprocess the data
grid = grid_generator()
data_processed, scaler, Tool_Mapping, onehot_encoder = grid_preprocess(grid)

# 2) read limited target data
path = '\Data\\'
filename = '8551_Limited_Sample.csv'

# data: for training initial model
# encoder: onehot encoder; currently no use
# Main: raw dataset; query data will be added to this dataset
Main = pd.read_csv(path + filename, index_col=0)
data, encoder = loadData(Main,onehot_encoder)

# 3) Specify test and sample size for splitting the dataset
source = data.copy()  # Source dataset including inputs and outputs

X_train_scaled_S, y_train_S, Scaler_S = dataprep_trainOnly(source, refScaler=scaler)

# 4) Testset
path = '\Data\\'
filename = '8551_Test_Sample.csv'
Test = pd.read_csv(path + filename, index_col=0)
test_data, encoder = loadDataCC2(Test,onehot_encoder)
X_test_scalded_S, y_test_S, Scaler_S = dataprep_trainOnly(test_data, refScaler=scaler)

# 5) Format normalized data to torch tensor
train_x = torch.from_numpy(X_train_scaled_S).float()
test_x = torch.from_numpy(X_test_scalded_S).float()

train_y = torch.from_numpy(y_train_S).float()
test_y = torch.from_numpy(y_test_S).float()

grid_x = torch.from_numpy(data_processed).float()
grid_y = torch.from_numpy(np.ones(grid_x.shape[0])).float()

# 6) Prepare data for training/iterations
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

grid_dataset = TensorDataset(grid_x, grid_y)
grid_loader = DataLoader(grid_dataset, batch_size=1024, shuffle=False)

# 7) Active learning via GP
n=40 # iteration

encoder_categories = ['TOOLING-Aluminium-606x-Null-v1', 'TOOLING-Composite-AS4-v1','TOOLING-Steel-1020-Null-v1',
                      'TOOLING-Invar-36-Null-v2']
encoder_categories_lower = [i.lower() for i in encoder_categories]

resin = 8551 # whether source or target - 8551, 8552

evaluation = []
for al in np.arange(1, n+1):

    # 7.a) train the model
    count = X_train_scaled_S.shape[0]
    le = np.arange(0, count)
    k = int(count/10)
    ch = choices(le, k=k)
    inducing_points = train_x[ch, :]
    model = GPModel(inducing_points=inducing_points)
    likelihood = gpytorch.likelihoods.BernoulliLikelihood()

    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

    num_epochs = 100

    model.train()
    likelihood.train()

    # We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
        {'params': likelihood.parameters()},
    ], lr=0.01)

    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

    epochs_iter = tqdm.autonotebook.tqdm(range(num_epochs), desc="Epoch")
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        minibatch_iter = tqdm.autonotebook.tqdm(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            minibatch_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()
            
    # 7.b) Evaluate the model
    evaluation.append(evaluate())
    print(evaluate())
    
    # 7.c)Query data
    #Find arg of n most uncertain poitns
    n = 5
    arg_query = active_query(n)
    # Generating the input array to be queried 
    new_inp = generate_input(arg_query, scaler, Tool_Mapping, onehot_encoder)
    # Reformat the tool type to be read by RAVEN properly
    dict_tool = dict(zip(encoder_categories_lower, encoder_categories))
    new_inp[:,0] = [dict_tool[i] for i in new_inp[:,0]]

    # 7.d) RAVEN simulation
    ravenAPI.HOST = 'localhost'
    ravenAPI.PORT = 58009
    for j in np.arange(new_inp.shape[0]):
        RAVEN(new_inp[j], j, al, resin, dire_update)
        
    # 4) Update training dataset
    Main, ext = load_main_data(dire_update, Main, al)
    print(Main.shape)
    data, encoder = loadDataCC2(Main,onehot_encoder)
    X_train_scaled_S, y_train_S, Scaler_S = dataprep_trainOnly(data, refScaler=scaler)
    train_x = torch.from_numpy(X_train_scaled_S).float()
    train_y = torch.from_numpy(y_train_S).float()
    train_dataset = TensorDataset(train_x, train_y)
    if Main.shape[0] > 256:
        train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    elif Main.shape[0] > 128:
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    else:
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        

### 4) Transfer learning

In [ ]:
# 1) load Target data (provided by AL)
path = '\Data\\'
filename = '8551_Limited_TrainingSet_Sampe.csv'
Main_T = pd.read_csv(path + filename, index_col=0)
data_T, encoder = loadDataCC2(Main_T,onehot_encoder)
X_train_T, X_valid_T, y_train_T, y_valid_T, scaler_s = dataprep(data_T, 0.2,refScaler=scaler)

# 2) TL with Freezing
dict_hist = {}
dict_preds = {}
patience = 100
lr=0.01
optimizer = tf.keras.optimizers.Adam(lr=lr)
callback = tf.keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True)
for i in range(5):
    model_TL_raw = tf.keras.models.load_model('Source_model.h5')
    model_1 = tf.keras.models.Sequential(model_TL_raw.layers[1].layers[:-1])
    model_1.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    model_TL = tf.keras.models.Sequential([model_TL_raw.layers[0], model_1])
    
    # Step1: Train TL model with freezing
    lr=0.01
    model_TL.layers[0].trainable = False
    for layer in model_TL.layers[1].layers[:-1]:
        layer.trainable = False
    model_TL.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    patience = 100
    history_TL = model_TL.fit(x=X_train_T, y=y_train_T, epochs=10, batch_size=16,
                                      validation_data=(X_valid_T, y_valid_T), callbacks=[callback])
    
    # Step2: unfreeze and train
    lr = 0.001
    model_TL.layers[0].trainable = True
    for layer in model_TL.layers[1].layers[:-1]:
        layer.trainable = True
    model_TL.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    patience = 100
    history_TL_2 = model_TL.fit(x=X_train_T, y=y_train_T, epochs=100, batch_size=16,
                                      validation_data=(X_valid_T, y_valid_T), callbacks=[callback])
    
    # Step3: Generate accuracy dicts
    dict_hist['acc{}'.format(str(i))] = history_TL.history['accuracy'] + history_TL_2.history['accuracy']
    dict_hist['val{}'.format(str(i))] = history_TL.history['val_accuracy'] + history_TL_2.history['val_accuracy']
    
    # Step4: Generate predictions (for roc curve)
    dict_preds[i] = model_TL.predict(X_test_T)
    
    # Step5: Performance metric resutls
    if i ==0:
        a_all = np.array(evaluate(model_TL, X_test_T, y_test_T))
    else:
        a_all = np.row_stack((a_all, evaluate(model_TL, X_test_T, y_test_T)))
        
mean_all = a_all.mean(axis=0)
std_all = a_all.std(axis=0)